In [1]:
%load_ext autotime

In [2]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

time: 919 µs


In [3]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 2.36 s


In [4]:
mg.init_jvm()

True

time: 103 ms


In [5]:
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv',
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.22 s


In [6]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')


time: 62.3 ms


In [7]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])


time: 196 ms


In [8]:
C.to_csv('books_candset.csv')

True

time: 4.29 ms


In [9]:
# Read candidate set 
C = mg.read_csv('books_candset.csv', ltable=wal, rtable=bwk)

time: 3.1 ms


In [10]:
#plan

# 1. sample candidate set --> S
# 2. label S --> L
# 3. Two matching options
#    a. ML matcher + Trigger
#    b. Rule-based matcher + trigger
# 3b. Rule-based matcher + trigger
#   1. Write matching rules
#   2. Add triggers
# 4. Choose one of the above two options (which ever produces better result), and 
#     apply it over candidate set

time: 1.64 ms


In [11]:
# sample table
S = mg.sample_table(C, 50)

time: 2.02 ms


In [12]:
# label table
#L = mg.label_table(S, 'gold')

time: 546 µs


In [13]:
# for demo reading a pre-labeled file
L = mg.read_csv('label_table_demo_books.csv', ltable=wal, rtable=bwk)

time: 3.77 ms


In [14]:
# ML matcher + trigger
#   1. Find best possible ML matcher
#       1. Get features for matching
#       2. Extract feature vectors
#       3. Select matcher using cross validation
#       4. Debug matcher
#       5. Get the best possible matcher
#   2. Add Triggers 

time: 1.47 ms


In [15]:
# get features for matching
feature_table = mg.get_features_for_matching(wal, bwk)

time: 3.11 s


In [16]:
#feature_table

time: 668 µs


In [17]:
# select a subset of matchers
f = feature_table.ix[[3,7,18,26, 53]]

time: 1.57 ms


In [18]:
f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
3,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x11f...,from magellan.feature.simfunctions import *\nf...
7,author_author_jac_qgm_3_qgm_3,author,author,qgm_3,qgm_3,jaccard,<function author_author_jac_qgm_3_qgm_3 at 0x1...,from magellan.feature.simfunctions import *\nf...
18,binding_binding_jac_qgm_3_qgm_3,binding,binding,qgm_3,qgm_3,jaccard,<function binding_binding_jac_qgm_3_qgm_3 at 0...,from magellan.feature.simfunctions import *\nf...
26,publisher_publisher_jac_qgm_3_qgm_3,publisher,publisher,qgm_3,qgm_3,jaccard,<function publisher_publisher_jac_qgm_3_qgm_3 ...,from magellan.feature.simfunctions import *\nf...
53,lang_lang_lev,lang,lang,None,None,lev,<function lang_lang_lev at 0x128506848>,from magellan.feature.simfunctions import *\nf...


time: 15.8 ms


In [19]:
G = mg.extract_feat_vecs(L, feature_table=f, attrs_after='gold')

time: 363 ms


In [20]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
3,3,15280600,9780217455008,0.720000,0.666667,1,1.000000,1,1
4,4,15286798,9780217825382,1.000000,1.000000,1,1.000000,1,1
5,5,15295588,9780217940207,0.308271,0.695652,1,0.083333,1,0
6,6,15334872,9780217270052,1.000000,1.000000,1,1.000000,1,1
7,7,15338150,9780217304399,0.280702,1.000000,1,1.000000,1,1
8,8,1566948,9780070479081,0.822222,0.260870,1,0.303030,1,1
9,9,4564015,9780080446837,1.000000,NaN,1,0.512821,1,0


time: 32.7 ms


In [21]:
# impute values
G.fillna(0, inplace=True)

time: 1.88 ms


In [22]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
3,3,15280600,9780217455008,0.720000,0.666667,1,1.000000,1,1
4,4,15286798,9780217825382,1.000000,1.000000,1,1.000000,1,1
5,5,15295588,9780217940207,0.308271,0.695652,1,0.083333,1,0
6,6,15334872,9780217270052,1.000000,1.000000,1,1.000000,1,1
7,7,15338150,9780217304399,0.280702,1.000000,1,1.000000,1,1
8,8,1566948,9780070479081,0.822222,0.260870,1,0.303030,1,1
9,9,4564015,9780080446837,1.000000,0.000000,1,0.512821,1,0


time: 15.7 ms


In [23]:
# Create ML matchers
dt = mg.DTMatcher(name='DecisionTree')
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF')
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

time: 3 ms


In [24]:
dt.name

'DecisionTree'

time: 1.82 ms


In [25]:
svm.name

'SVM_25705057710427006441'

time: 2.31 ms


In [26]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision') 
        # metric can take precision, recall, f1

time: 101 ms


In [27]:
result['selected_matcher']

time: 4.39 ms


In [28]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,1.000000,1.000000,0.857143,0.777778,0.625,0.851984
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.833333,0.750000,1.000000,0.714286,1.000,0.859524
2,SVM_25705057710427006441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.666667,0.888889,0.600000,1.000000,0.600,0.751111
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.666667,0.700000,0.777778,0.800000,0.800,0.748889
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.750000,0.750000,0.777778,0.888889,0.900,0.813333


time: 26 ms


In [29]:
# debug: understand why DT matcher is not performing well

# Split feature vectors to train and test
train_test = mg.train_test_split(G, train_proportion=0.7, random_state=0)
train = train_test['train']
test = train_test['test']

time: 5.62 ms


In [30]:
mg.vis_debug_dt(dt, train, test, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold') 

time: 2.81 s


In [31]:
mg.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold') 

time: 1.01 s


In [32]:
# Try adding all the features
H = mg.extract_feat_vecs(L, feature_table=feature_table, attrs_after='gold')

time: 707 ms


In [33]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision') 

time: 68.6 ms


In [34]:
result['selected_matcher']

time: 2.65 ms


In [35]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.800000,0.888889,0.857143,0.833333,0.857143,0.847302
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,1.000000,0.857143,0.750000,0.888889,0.750000,0.849206
2,SVM_25705057710427006441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.500000,1.000000,0.666667,0.800000,0.700000,0.733333
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.900000,0.900000,0.500000,0.666667,0.700000,0.733333
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.777778,0.888889,0.666667,0.875000,0.888889,0.819444


time: 21.5 ms


In [36]:
# Choose DT matcher (with subset of features)

# Add triggers to DT Matcher
# Plan
# Split G
# G :------------------|----------------------------|--------------------|
#   [development(60%): train(30%)  + validation(30%)]     [test(40%)]
          
# Use 'development' set to come up with triggers
# Use 'test' set to evaluate ML matcher + Trigger

time: 1.95 ms


In [37]:
development_test = mg.train_test_split(G, train_proportion=0.6, 
                                       random_state=0)
development = development_test['train']
test = development_test['test']

time: 5.13 ms


In [38]:
len(development), len(test)

(30, 20)

time: 2.5 ms


In [39]:
development.head(2)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
15,15,5087512,9780028348261,0.534483,0.6,1,0.264706,1,1
30,30,14022273,9780217263016,0.408759,1.0,1,1.000000,1,1


time: 10.6 ms


In [40]:
mg.vis_debug_dt(dt, development, test,
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 3.69 s


In [41]:
train_validation = mg.train_test_split(development, train_proportion=0.5, 
                                       random_state=0)

time: 3.02 ms


In [42]:
train = train_validation['train']
validation = train_validation['test']

time: 1.24 ms


In [43]:
dt.fit(table=train, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 6.27 ms


In [44]:
I = dt.predict(table=validation, 
              exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
              target_attr='predicted', append=True, inplace=False)

time: 20 ms


In [45]:
eval_summary = mg.eval_matches(I, 'gold', 'predicted')

time: 8.92 ms


In [46]:
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7272727272727273, 1.0, 0.8421052631578948, 0.0, 3.0)

time: 2.22 ms


In [47]:
# Add triggers to address false negatives
# if authors match exactly then declare match
pos_trigger = mg.MatchTrigger()
pos_trigger.add_cond_rule('author_author_lev(ltuple, rtuple) == 1', 
                          feature_table=feature_table)
pos_trigger.add_cond_status(True)
pos_trigger.add_action(1)


True

time: 3.43 ms


In [48]:
I_ = pos_trigger.execute(I, 'predicted', inplace=False)

time: 512 ms


In [49]:
eval_summary = mg.eval_matches(I, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7272727272727273, 1.0, 0.8421052631578948, 0.0, 3.0)

time: 13.5 ms


In [50]:
# evaluate ml matcher + match trigger
# train with development data
dt.fit(table=development, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 5.12 ms


In [51]:
J = dt.predict(table=test, 
              exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
              target_attr='predicted',
              append=True, inplace=False)

time: 21.3 ms


In [52]:
eval_summary = mg.eval_matches(J, 'gold', 'predicted')

time: 9.16 ms


In [53]:
eval_summary.keys()

['prec_numerator',
 'prec_denominator',
 'precision',
 'recall_numerator',
 'recall_denominator',
 'recall',
 'f1',
 'pred_pos_num',
 'false_pos_num',
 'false_pos_ls',
 'pred_neg_num',
 'false_neg_num',
 'false_neg_ls']

time: 2.45 ms


In [54]:
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.9333333333333333, 0.8235294117647058, 0.8749999999999999, 3.0, 1.0)

time: 2.33 ms


In [55]:
K = pos_trigger.execute(J, 'predicted', inplace=False)

time: 500 ms


In [56]:
K

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold,predicted
0,28,15368811,9780217110952,0.852941,1.000000,1,1.000000,1,1,1
1,11,5681971,9780073380339,0.807018,0.413793,0,0.200000,1,1,1
2,10,5514791,9780114958367,0.878049,0.000000,0,0.000000,0,0,0
3,41,10948704,9780131077744,0.333333,0.000000,0,0.000000,1,0,0
4,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1,1
5,27,15348586,9780217521376,1.000000,1.000000,1,1.000000,1,1,1
6,38,1581211,9780201738278,0.919355,1.000000,1,0.785714,1,1,1
7,31,14022804,9780217519724,0.791667,1.000000,1,1.000000,1,1,1
8,22,3618742,9780060837563,1.000000,1.000000,1,0.320000,1,1,1
9,4,15286798,9780217825382,1.000000,1.000000,1,1.000000,1,1,1


time: 41.2 ms


In [57]:
eval_summary = mg.eval_matches(K, 'gold', 'predicted')

time: 8.77 ms


In [58]:
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.9444444444444444, 1.0, 0.9714285714285714, 0.0, 1.0)

time: 2.17 ms


In [59]:
#--------Rule-based matcher + triggers--------#
# 1. write rules
# 2. add triggers

time: 985 µs


In [60]:
rm = mg.BooleanRuleMatcher()
rm.add_rule(['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.6' 
             ], 
            feature_table=feature_table)


True

time: 3.28 ms


In [61]:
rm.add_rule(['author_author_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.6'], 
            feature_table=f)
rm.add_rule(['binding_binding_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.5'], 
            feature_table=f)

True

time: 4.6 ms


In [62]:
# Note here the input in candidate set !!!
X = rm.predict(table=L, target_attr='predicted', 
               append=True, inplace=False)

time: 523 ms


In [63]:
X.head()

,_id,ltable.id,rtable.id,ltable.title,ltable.author,rtable.title,rtable.author,gold,predicted
0,0,4086892,9780226156439,The Bedtrick: Tales of Sex and Masquerade,"Doniger, Wendy",Bedtrick : Tales of Sex and Masquerade,"Doniger, Wendy",1,1
1,1,535599,9780060655808,HarperCollins Study Bible-NRSV,"Meeks, Wayne A.",Harpercollins Study Bible Standard Version wit...,"Meeks, Wayne A.",1,1
2,3,1386538,9780130901262,Adaptive Filter Theory,"Haykin, Simon S.",Adaptive Filter Theory,"Haykin, Simon",1,1
3,5,15280600,9780217455008,"Catalogue of Earthquakes on the Pacific Coast,...","McAdie, Alexander","Catalogue of Earthquakes on the Pacific Coast,...","Mcadie, Alexander",1,1
4,6,15286798,9780217825382,"Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill","Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill",1,1


time: 30.4 ms


In [64]:
eval_summary = mg.eval_matches(X, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7391304347826086, 0.9444444444444444, 0.8292682926829269, 2.0, 12.0)

time: 10.3 ms


In [65]:
mg.vis_debug_rm(rm, L, 'gold', feature_table)

time: 16.3 s


In [66]:
# address false positives
# Add triggers to address false negatives
# if authors match exactly then declare match
neg_trigger = mg.MatchTrigger()
neg_trigger.add_cond_rule(['lang_lang_lev(ltuple, rtuple) > 0.5'],
    feature_table=feature_table)
neg_trigger.add_cond_status(False)
neg_trigger.add_action(0)


True

time: 3.41 ms


In [67]:
Y = neg_trigger.execute(X, 'predicted', inplace=False)

time: 518 ms


In [68]:
eval_summary = mg.eval_matches(Y, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7727272727272727, 0.9444444444444444, 0.85, 2.0, 10.0)

time: 11.8 ms


In [69]:
# choose ml matcher + trigger

time: 591 µs


In [70]:
M = mg.extract_feat_vecs(C, feature_table=f)

time: 375 ms


In [71]:
len(M)

68

time: 1.67 ms


In [72]:
dt.fit(table=G, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 5.41 ms


In [73]:
M.fillna(0, inplace=True)

time: 1.52 ms


In [74]:
N = dt.predict(table=M, exclude_attrs=['_id', 'ltable.id', 'rtable.id'],
              target_attr='predicted', append=True, inplace=False)

time: 7.12 ms


In [75]:
N.head()

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,predicted
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,12537042,9780030402432,0.243243,0.285714,0,0.025641,0,1
3,3,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
4,4,13945932,9780205700424,0.375000,0.375000,0,0.733333,1,1


time: 12.9 ms
